# Geoglyph Viewer with ipywidgets

Interactive widget-based viewer for geoglyph metadata and images.

## 1. Import Required Libraries

Import necessary libraries including ipywidgets, json, PIL, and IPython.display for image rendering.

In [1]:
import io
import json
import os
import sys
from pathlib import Path
from typing import Optional

import numpy as np
import ipywidgets as widgets
from IPython.display import display
from PIL import Image as PILImage

# Add parent directory to path to import format module
sys.path.insert(0, os.path.dirname(os.path.abspath('.')))
from format import make_random_crops, make_fixed_crops, make_polygon_thresholds_crops, fill_with_noise

# ============================================================================
# DEFAULT CONSTANTS
# ============================================================================
DEFAULT_SEED = 42
DEFAULT_STRIDE = 32
DEFAULT_WINDOW_SIZE = 128
DEFAULT_NOISE_LEVEL = 0.0
DEFAULT_THRESHOLD = 0.1
DEFAULT_IMAGE_TYPE = "overlay"
DEFAULT_MAX_DISPLAY_CROPS = 20
MAX_COMPARISON_CROPS = 40
CROPS_PER_ROW = 5
GRID_BACKGROUND_COLOR = (240, 240, 240)
IMAGE_KEYS = {"overlay": "overlay_jpeg", "ortho": "ortho_jpeg"}
METADATA_MISSING_VALUE = "?"


## 2. Load and Parse JSON Metadata

Create a function to load and parse the JSON metadata file.

In [2]:
def load_metadata(meta_path: str) -> dict:
    """Load metadata JSON file."""
    with open(meta_path, "r", encoding="utf-8") as f:
        return json.load(f)


def resolve_image_path(meta_path: str, files: dict, image_type: str = DEFAULT_IMAGE_TYPE) -> str:
    """Resolve the full path to the geoglyph image."""
    base_dir = os.path.dirname(os.path.abspath(meta_path))
    
    # Try primary type, then fallback to any available
    key = IMAGE_KEYS.get(image_type)
    if key and key in files:
        return os.path.join(base_dir, files[key])
    
    for key in ["overlay_jpeg", "ortho_jpeg"]:
        if key in files:
            return os.path.join(base_dir, files[key])
    
    raise FileNotFoundError("No suitable JPEG image found in metadata 'files'.")


def _extract_polygon_vertices(metadata: dict) -> Optional[list]:
    """Extract polygon vertices from common metadata keys."""
    if not metadata:
        return None
    if "polygon_points" in metadata:
        polygon_points = metadata["polygon_points"]
        if polygon_points and isinstance(polygon_points, list) and isinstance(polygon_points[0], dict) and "exterior" in polygon_points[0]:
            coords = polygon_points[0]["exterior"]
        else:
            coords = polygon_points
    elif "polygon" in metadata:
        coords = metadata["polygon"]
    elif "geometry" in metadata:
        geom = metadata["geometry"]
        if isinstance(geom, dict) and "coordinates" in geom:
            coords = geom["coordinates"]
            if coords and isinstance(coords[0], (list, tuple)) and isinstance(coords[0][0], (list, tuple)):
                coords = coords[0]
        else:
            coords = geom
    elif "coordinates" in metadata:
        coords = metadata["coordinates"]
    else:
        return None
    try:
        return [(float(x), float(y)) for x, y in coords]
    except Exception:
        return None


def _polygon_geo_to_pixel(vertices: list, metadata: dict) -> Optional[list]:
    """Convert geographic polygon vertices to pixel coordinates if bounds and image_shape exist."""
    bounds = metadata.get("bounds") if metadata else None
    img_shape = metadata.get("image_shape") if metadata else None
    if not bounds or not img_shape:
        return vertices
    geo_minx = bounds.get("minx")
    geo_miny = bounds.get("miny")
    geo_maxx = bounds.get("maxx")
    geo_maxy = bounds.get("maxy")
    width = img_shape.get("width")
    height = img_shape.get("height")
    if None in (geo_minx, geo_miny, geo_maxx, geo_maxy, width, height) or width == 0 or height == 0:
        return vertices

    def geo_to_pixel(lon, lat):
        x_norm = (lon - geo_minx) / (geo_maxx - geo_minx) if geo_maxx != geo_minx else 0.0
        y_norm = (geo_maxy - lat) / (geo_maxy - geo_miny) if geo_maxy != geo_miny else 0.0
        return (x_norm * width, y_norm * height)

    try:
        return [geo_to_pixel(lon, lat) for lon, lat in vertices]
    except Exception:
        return None


def load_polygon_vertices_from_metadata(metadata: dict) -> Optional[list]:
    """Extract polygon vertices and map to pixel coordinates when possible."""
    raw_vertices = _extract_polygon_vertices(metadata)
    if not raw_vertices:
        return None
    pixel_vertices = _polygon_geo_to_pixel(raw_vertices, metadata)
    return pixel_vertices if pixel_vertices else raw_vertices


## 3. Create Image Display Function

Implement a function that displays the geoglyph image and metadata information.

## 3a. Crop Visualization Helper

Create a function to overlay crop regions on the image.

In [3]:
def _convert_to_uint8_image(crop_array: np.ndarray) -> np.ndarray:
    """Convert array to uint8 format, handling various input types."""
    if not isinstance(crop_array, np.ndarray):
        return crop_array
    if crop_array.dtype != np.uint8:
        crop_array = np.clip(crop_array, 0, 255).astype(np.uint8)
    if len(crop_array.shape) == 3 and crop_array.shape[2] == 4:
        crop_array = crop_array[:, :, :3]  # Remove alpha channel
    return crop_array


def create_crops_grid(image_array: np.ndarray, crop_method: str, window_size: int = DEFAULT_WINDOW_SIZE, 
                      n_crops: int = DEFAULT_MAX_DISPLAY_CROPS, seed: int = DEFAULT_SEED, 
                      stride: int = DEFAULT_STRIDE, threshold: float = DEFAULT_THRESHOLD, 
                      polygon_vertices: Optional[list] = None, noise_type: Optional[str] = None, 
                      noise_level: float = DEFAULT_NOISE_LEVEL) -> PILImage.Image:
    """Create a grid of cropped images with optional noise filling."""
    display_n_crops = min(n_crops, DEFAULT_MAX_DISPLAY_CROPS)

    # Generate crops with method-specific parameters
    if crop_method == "random":
        crops_list = make_random_crops(image_array, window_size, display_n_crops, seed=seed)
    elif crop_method == "polygon_threshold":
        if polygon_vertices is None:
            # Fallback to fixed crops if no polygon is available
            crops_list = make_fixed_crops(image_array, window_size, n_crops=n_crops, stride=stride)
        else:
            crops_list = make_polygon_thresholds_crops(
                image_array,
                polygon_vertices,
                window_size,
                n_crops=n_crops,
                stride=stride,
                threshold=threshold,
            )
    else:  # fixed
        crops_list = make_fixed_crops(image_array, window_size, n_crops=n_crops, stride=stride)

    # Randomly pick a subset for non-random methods when more crops exist than we want to display
    if crop_method in {"fixed", "polygon_threshold"} and len(crops_list) > display_n_crops:
        rng = np.random.default_rng(seed)
        selected_indices = rng.choice(len(crops_list), size=display_n_crops, replace=False)
        crops_list = [crops_list[idx] for idx in selected_indices]
    else:
        crops_list = crops_list[:display_n_crops]

    # Apply noise filling if specified
    if noise_type and noise_level > 0:
        crops_list = [
            (fill_with_noise(crop, mask, noise_level, noise_type, seed=seed + idx), mask)
            for idx, (crop, mask) in enumerate(crops_list)
        ]

    # Create grid layout
    num_rows = (len(crops_list) + CROPS_PER_ROW - 1) // CROPS_PER_ROW
    grid_size = (CROPS_PER_ROW * window_size, num_rows * window_size)
    grid_img = PILImage.new('RGB', grid_size, color=GRID_BACKGROUND_COLOR)

    # Paste crops into grid
    for idx, (crop, mask) in enumerate(crops_list):
        x = (idx % CROPS_PER_ROW) * window_size
        y = (idx // CROPS_PER_ROW) * window_size
        crop_uint8 = _convert_to_uint8_image(crop)
        crop_pil = PILImage.fromarray(crop_uint8)
        grid_img.paste(crop_pil, (x, y))

    return grid_img


In [4]:
def _pil_to_bytes(pil_img: PILImage.Image) -> bytes:
    """Convert PIL Image to bytes."""
    img_bytes = io.BytesIO()
    pil_img.save(img_bytes, format='JPEG')
    img_bytes.seek(0)
    return img_bytes.read()


def _create_metadata_html(poly_idx: str, clazz: str, image_type: str, crop_method: Optional[str] = None) -> str:
    """Generate metadata HTML."""
    crop_info = f"<p><b>Crop Method:</b> {crop_method}</p>" if crop_method else ""
    return f"""
    <div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; background-color: #f9f9f9; margin-top: 15px;">
        <h3>Geoglyph #{poly_idx}</h3><hr>
        <p><b>Class:</b> {clazz}</p>
        <p><b>Image Type:</b> {image_type}</p>
        {crop_info}
    </div>"""


def _get_crop_label(crop_method: str, n_crops: int, threshold: float = DEFAULT_THRESHOLD) -> str:
    """Generate label for crop display."""
    display_count = min(n_crops, MAX_COMPARISON_CROPS)
    if crop_method == "polygon_threshold":
        return f"Polygon Threshold (threshold: {threshold}, showing {display_count}/{n_crops})"
    return f"{crop_method.capitalize()} Crops (showing {display_count}/{n_crops})"


def display_geoglyph_with_crops(meta_path: str, image_type: str = DEFAULT_IMAGE_TYPE, 
                                crop_method: Optional[str] = None,
                                window_size: int = DEFAULT_WINDOW_SIZE, 
                                n_crops: int = DEFAULT_MAX_DISPLAY_CROPS, 
                                seed: int = DEFAULT_SEED, stride: int = DEFAULT_STRIDE,
                                threshold: float = DEFAULT_THRESHOLD, 
                                polygon_vertices: Optional[list] = None,
                                noise_type: Optional[str] = None, 
                                noise_level: float = DEFAULT_NOISE_LEVEL):
    """Display geoglyph image and crop visualization side by side."""
    metadata = load_metadata(meta_path)
    image_path = resolve_image_path(meta_path, metadata.get("files", {}), image_type)
    poly_idx = metadata.get("polygon_index", METADATA_MISSING_VALUE)
    clazz = metadata.get("class", METADATA_MISSING_VALUE)
    inferred_polygon = polygon_vertices or load_polygon_vertices_from_metadata(metadata)
    
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        return
    
    # Load image
    img_array = np.array(PILImage.open(image_path))
    
    # Handle comparison mode
    if crop_method == "comparison":
        return display_comparison(meta_path, image_type, img_array, window_size, n_crops, seed, threshold, 
                                 inferred_polygon, noise_type, noise_level)
    
    # Load original image for display
    with open(image_path, 'rb') as f:
        left_img_widget = widgets.Image(value=f.read(), format='jpg')
    
    # Generate crop grid if specified
    if crop_method:
        crops_grid = create_crops_grid(img_array, crop_method, window_size, n_crops, seed, stride, threshold, 
                                      inferred_polygon, noise_type, noise_level)
        right_img_widget = widgets.Image(value=_pil_to_bytes(crops_grid), format='jpg', width='100%')
        right_label = _get_crop_label(crop_method, n_crops, threshold)
    else:
        right_img_widget = left_img_widget
        right_label = "Original Image"
    
    # Display results
    display(widgets.HTML(_create_metadata_html(poly_idx, clazz, image_type, crop_method)))
    display(widgets.HBox([
        widgets.VBox([widgets.HTML("<b>Original Image</b>"), left_img_widget], layout=widgets.Layout(flex='1')),
        widgets.VBox([widgets.HTML(f"<b>{right_label}</b>"), right_img_widget], layout=widgets.Layout(flex='1'))
    ], layout=widgets.Layout(width='100%')))


def display_comparison(meta_path: str, image_type: str, img_array: np.ndarray, 
                      window_size: int, n_crops: int, seed: int, threshold: float, 
                      polygon_vertices: Optional[list],
                      noise_type: Optional[str] = None, noise_level: float = DEFAULT_NOISE_LEVEL):
    """Display comparison of crop methods side by side with original image."""
    metadata = load_metadata(meta_path)
    poly_idx = metadata.get("polygon_index", METADATA_MISSING_VALUE)
    clazz = metadata.get("class", METADATA_MISSING_VALUE)
    display_count = min(n_crops, MAX_COMPARISON_CROPS)
    inferred_polygon = polygon_vertices or load_polygon_vertices_from_metadata(metadata)
    
    # Create info HTML
    info_html = f"""
    <div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; background-color: #f9f9f9; margin-top: 15px;">
        <h3>Geoglyph #{poly_idx} - Crop Methods Comparison</h3><hr>
        <p><b>Class:</b> {clazz}</p>
        <p><b>Image Type:</b> {image_type}</p>
        <p><b>Window Size:</b> {window_size} | <b>Crops:</b> {n_crops}</p>
    </div>"""
    
    # Load original image for display
    image_path = resolve_image_path(meta_path, metadata.get("files", {}), image_type)
    with open(image_path, 'rb') as f:
        original_img_widget = widgets.Image(value=f.read(), format='jpg', width='100%')
    
    # Generate crops for each method
    methods = [("Random", "random"), ("Fixed", "fixed"), ("Polygon Threshold", "polygon_threshold")]
    grids = {name: create_crops_grid(img_array, method, window_size, n_crops, seed=seed, 
                                    threshold=threshold, polygon_vertices=inferred_polygon,
                                    noise_type=noise_type, noise_level=noise_level) 
             for name, method in methods}
    
    # Create comparison box with original image first
    comparison_items = [
        widgets.VBox([
            widgets.HTML("<b>Original Image</b>"),
            original_img_widget
        ], layout=widgets.Layout(flex='1'))
    ]
    
    for name, _ in methods:
        comparison_items.append(
            widgets.VBox([
                widgets.HTML(f"<b>{name} Crops<br>(showing {display_count}/{n_crops})</b>"),
                widgets.Image(value=_pil_to_bytes(grids[name]), format='jpg', width='100%')
            ], layout=widgets.Layout(flex='1'))
        )
    
    comparison_box = widgets.HBox(comparison_items, layout=widgets.Layout(width='100%'))
    
    display(widgets.HTML(info_html))
    display(comparison_box)


def display_geoglyph(meta_path: str, image_type: str = DEFAULT_IMAGE_TYPE):
    """Display geoglyph image and metadata information."""
    display_geoglyph_with_crops(meta_path, image_type)


## 4. Build Interactive Widget Interface

Create ipywidgets controls to navigate and view geoglyphs.

In [5]:
# Widget style constant
WIDGET_STYLE = {"description_width": "120px"}

crop_method_dropdown = widgets.Dropdown(
    options=[("None", None), ("Random", "random"), ("Fixed", "fixed"), ("Polygon Threshold", "polygon_threshold"), ("Comparison", "comparison")],
    value=None,
    description="Crop Method:",
    style=WIDGET_STYLE
)

In [6]:
# Create widgets with consistent styling
metadata_path_input = widgets.Text(
    value="test_geos/unita_geoglif_0000_metadata.json",
    placeholder="Enter metadata file path",
    description="Metadata Path:",
    style=WIDGET_STYLE,
    layout=widgets.Layout(width="100%")
)

image_type_dropdown = widgets.Dropdown(
    options=[("Overlay", "overlay"), ("Ortho", "ortho")],
    value="overlay",
    description="Image Type:",
    style=WIDGET_STYLE
)

window_size_input = widgets.IntSlider(
    value=128, min=32, max=512, step=32,
    description="Window Size:",
    style=WIDGET_STYLE
)

n_crops_input = widgets.IntSlider(
    value=16, min=1, max=100, step=1,
    description="Number of Crops:",
    style=WIDGET_STYLE
)

crop_seed_input = widgets.IntText(
    value=42,
    description="Random Seed:",
    style=WIDGET_STYLE
)

stride_input = widgets.IntSlider(
    value=32, min=1, max=256, step=1,
    description="Stride:",
    style=WIDGET_STYLE
)

threshold_input = widgets.FloatSlider(
    value=0.1, min=0.0, max=1.0, step=0.01,
    description="Threshold:",
    style=WIDGET_STYLE
)

noise_type_dropdown = widgets.Dropdown(
    options=[("None", None), ("Gaussian", "gaussian"), ("Uniform", "uniform")],
    value=None,
    description="Noise Type:",
    style=WIDGET_STYLE
)

noise_level_input = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.1,
    description="Noise Level:",
    style=WIDGET_STYLE
)

load_button = widgets.Button(
    description="Load Geoglyph",
    button_style="info",
    tooltip="Click to load and display the geoglyph"
)

output_area = widgets.Output()

In [7]:
# Define button click handler
def on_load_click(b):
    output_area.clear_output(wait=True)
    with output_area:
        if not metadata_path_input.value.strip():
            print("Please enter a metadata file path.")
            return
        
        display_geoglyph_with_crops(
            metadata_path_input.value,
            image_type=image_type_dropdown.value,
            crop_method=crop_method_dropdown.value,
            window_size=window_size_input.value,
            n_crops=n_crops_input.value,
            seed=crop_seed_input.value,
            stride=stride_input.value,
            threshold=threshold_input.value,
            noise_type=noise_type_dropdown.value,
            noise_level=noise_level_input.value
        )

load_button.on_click(on_load_click)

## 5. Display Geoglyph with Metadata Information

Enter the path to a metadata JSON file and click "Load Geoglyph" to view the image and metadata.

In [8]:
# Layout the widgets
control_box = widgets.VBox([
    metadata_path_input,
    widgets.HBox([image_type_dropdown, load_button]),
    widgets.HTML("<hr><b>Crop Visualization Settings:</b>"),
    crop_method_dropdown,
    widgets.HBox([window_size_input, n_crops_input]),
    widgets.HBox([stride_input, threshold_input]),
    crop_seed_input,
    widgets.HTML("<hr><b>Noise Filling Settings:</b>"),
    widgets.HBox([noise_type_dropdown, noise_level_input])
])

# Display the interface
display(control_box)
display(output_area)

Output()